# [Learn Quantum Computing with Python and Q#](https://www.manning.com/books/learn-quantum-computing-with-python-and-q-sharp?a_aid=learn-qc-granade&a_bid=ee23f338)<br>Chapter 11 Exercise Solutions
----
> Copyright (c) Sarah Kaiser and Chris Granade.
> Code sample from the book "Learn Quantum Computing with Python and Q#" by
> Sarah Kaiser and Chris Granade, published by Manning Publications Co.
> Book ISBN 9781617296130.
> Code licensed under the MIT License.


### Exercise 11.1 

**What are the powers of $11$ when computed$\mod 21$?**

In [1]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Logical;

function Powers(generator : Int, modulus : Int) : Int[] {
    mutable powers = [1];
    mutable acc = generator;
    
    while (acc != 1) {
        set powers += [acc];
        set acc = acc * generator % modulus;
    }
    
    return powers;
}

function PowersOf11Mod21(): Int[] {
    return Powers(11, 21);
}

Powers, PowersOf11Mod21

In [2]:
%simulate PowersOf11Mod21

1, 11, 16, 8, 4, 2

**How long does it take to loop back around to $11^0 = 1$?**

It takes $11^6 = 1$ before the powers of $11$ loop back around, so we say that the period of the function $f(x) = 11^n \mod 21$ is 6.
That is, $f(x) = f(x + 6)$ for all $x$, and $6$ is the smallest number for which this is the case.

**Does it matter if you take the modulus by $21$ at the end, or whether you compute the modulus at each step?**

No, you can multiply your accumulated value by 11 and then take it mod 21 and then repeat that as many times as needed, and it is the same as modding at the end. The incremental approach scales better as you don't have to store such large numbers and then mod them.

----
### Exercise 11.2

**Using either Python or Q#, try showing that one or both of the potential factor candidates from the last step of Shor's algorithm share factors with the modulus. Use $N = 143$, $g = 19$, and the period $r = 60$.**

Recall step 6 of Shor's Algorithm is:
> 6. Either $g^{\frac{r}{2}} - 1$ or $g^{\frac{r}{2}} + 1$ shares a factor with $N$.

Thus, you'll want to confirm that either $g^{r / 2} - 1 \bmod N$ or $g^{r / 2} + 1 \bmod N$ shares a factor with $N$; that is, isn't coprime with $N$.
In Q#, it's easy to check if two integers are coprime by using the [`IsCoprimeI` function](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.math.iscoprimei).

In [3]:
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

function SharedFactorsFact(generator : Int, modulus : Int, period : Int) : Unit {
    let halfPower = ExpModI(generator, period / 2, modulus);
    Fact(
        not IsCoprimeI(modulus, halfPower + 1 % modulus) or
        not IsCoprimeI(modulus, halfPower - 1 % modulus),
        $"Neither {generator}^({period} / 2) + 1 or {generator}^({period} / 2) - 1 shares a factor with {modulus}."
    );
    
}

function ExerciseFact() : Unit {
    SharedFactorsFact(19, 143, 60);
}

ExerciseFact, SharedFactorsFact

In [4]:
%simulate ExerciseFact

()

----
### Exercise 11.3

**What's the GCD of 35 and 30?**

In [5]:
open Microsoft.Quantum.Math;
function ExerciseGCD() : Int {
    return GreatestCommonDivisorI (35, 30); 
}

ExerciseGCD

In [6]:
%simulate ExerciseGCD

5

This does help you find the factors of $35$ because now you know that $5$ is a factor in common between $35$ and $30$ and thus is also a factor of $35$. You can then divide $35$ by $5$ and get that $7$ is also a factor of $35$.

----
### Exercise 11.4

**What state would your register be in after multiplying by 5 modulo 9?**

> Note: the code below is the same sample code that is in `IntegerFactorization.ipynb`.

In [7]:
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

operation PrepareSuperpositionOfTwoInts(register : LittleEndian, intPair : (Int, Int)) 
: Unit is Adj + Ctl {
    using(ctrl = Qubit()) {
        H(ctrl);
        
        within{
            X(ctrl);
        } 
        apply{
            Controlled ApplyXorInPlace([ctrl], (Fst(intPair), register));
        }

        Controlled ApplyXorInPlace([ctrl], (Snd(intPair), register));
        (ControlledOnInt(Snd(intPair), Y))(register!, ctrl);  
    }
}


operation MultiplyInSuperpositionMod(
    superpositionInts : (Int, Int), 
    multiplier : Int, 
    modulus : Int
) 
: Int {

    using (target = Qubit[BitSizeI(modulus)]) {

        PrepareSuperpositionOfTwoInts(LittleEndian(target), superpositionInts);
        
        Message("Before multiplication:");
        DumpMachine();
        
        MultiplyByModularInteger(multiplier, modulus, LittleEndian(target));
        
        Message("After multiplication:");
        DumpMachine();
        return MeasureInteger(LittleEndian(target));

    }
}

operation MultiplyInSuperpostionExercise4() :  Int {
    return MultiplyInSuperpositionMod((2, 7), 5, 9);
}

MultiplyInSuperpositionMod, MultiplyInSuperpostionExercise4, PrepareSuperpositionOfTwoInts

In [8]:
%simulate MultiplyInSuperpostionExercise4

Before multiplication:


|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0.7071067811865476 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	0 + -0.7071067811865476𝑖
|8⟩	0 + 0𝑖
|9⟩	0 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0 + 0𝑖
|12⟩	0 + 0𝑖
|13⟩	0 + 0𝑖
|14⟩	0 + 0𝑖
|15⟩	0 + 0𝑖

After multiplication:


|0⟩	3.467175960142825E-32 + 2.1023771756815435E-32𝑖
|1⟩	0.7071067811865541 + -5.418806267113754E-16𝑖
|2⟩	2.294456672237141E-32 + 7.192203822658452E-33𝑖
|3⟩	8.247755967606319E-33 + 1.1063934099580699E-32𝑖
|4⟩	-1.4080597125543172E-31 + -9.685086345191832E-32𝑖
|5⟩	-7.131093317436198E-32 + 9.294993713809864E-32𝑖
|6⟩	-7.436614165259295E-32 + 1.268469152811301E-32𝑖
|7⟩	-8.263810413191576E-32 + 6.262887267319684E-32𝑖
|8⟩	-7.04205339859851E-16 + -0.7071067811865552𝑖
|9⟩	-1.3230368779675594E-32 + 2.1381750218189806E-32𝑖
|10⟩	3.2671589541784516E-17 + 3.4390361378065145E-17𝑖
|11⟩	-8.297768771734271E-33 + -1.1467293963870534E-32𝑖
|12⟩	-1.1275880242874212E-32 + 9.32231042446029E-33𝑖
|13⟩	-9.914050009022099E-33 + 2.1612634281727344E-32𝑖
|14⟩	-1.6096538217897548E-33 + -1.1980190880341418E-32𝑖
|15⟩	-1.3703392235287878E-33 + 4.290813284059111E-33𝑖

1

You can see that after multiplication, the state is $\frac{1}{\sqrt{2}}\left(\left|1\right\rangle + \left|8\right\rangle\right)$.

----
### Exercise 11.5

**Try to multiply by 3 modulo 9, you'll get an error; why?**

In [9]:
operation MultiplyInSuperpostionExercise5() :  Int {
    // Here we are multiplying 3 by a superposition of 2 and 
    // 3, all mod 9.
    return MultiplyInSuperpositionMod((2, 7), 3, 9);
}

MultiplyInSuperpostionExercise5

In [10]:
%simulate MultiplyInSuperpostionExercise5

Before multiplication:


|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0.7071067811865476 + 0𝑖
|3⟩	0 + 0𝑖
|4⟩	0 + 0𝑖
|5⟩	0 + 0𝑖
|6⟩	0 + 0𝑖
|7⟩	0 + -0.7071067811865476𝑖
|8⟩	0 + 0𝑖
|9⟩	0 + 0𝑖
|10⟩	0 + 0𝑖
|11⟩	0 + 0𝑖
|12⟩	0 + 0𝑖
|13⟩	0 + 0𝑖
|14⟩	0 + 0𝑖
|15⟩	0 + 0𝑖

Source,Callable
/workspaces/code/ch11/D:/a/1/s/submodules/QuantumLibraries/Standard/src/Arithmetic/Modular.qs:0,Microsoft.Quantum.Arithmetic.MultiplyByModularInteger
/snippet_da8477ab-bc30-45fc-b3ef-2279d5725c6a.qs:0,MultiplyInSuperpositionMod
(notebook),MultiplyInSuperpostionExercise5


False ≠ True: `constMultiplier` and `modulus` must be co-prime


<details>
    <summary>Explanation</summary>
    Multiplying by $3 \mod 9$ is not reversible.
    For instance, both $1 \times 3$ and $4 \times 3$ mod 9 give 0, even though $1 \ne 4 \mod 9$.
    Since classical functions have to be reversible in order to be represented by quantum operations, the <code>MultiplyByModularInteger</code> operation raises an error in this case.
</details>
   

----
### Epilogue

_The following cell logs what version of the components this was last tested with._

In [ ]:
%version